# Prompt Engineering with GPT and LangChain

LangChain is framework that is extremely helpful for prompt engineering and the integration of generative AI capabilities in applications or data platforms. It has many capabilities, some of which will not be introduced until later modules, but we will start with a gentle introduction to some of the easy-to-understand concepts in the framework.

You'll build an AI agent that uses Python and GPT to perform sentiment analysis on financial headlines.

In more detail, you'll cover:
- Getting set up with an OpenAI developer account and integration with Workspace.
- Interacting with OpenAI models through the langchain framework.
- Using prompt templates that write reusable, dynamic prompts.
- Working with LLM chains.
- Automatically parsing the output of an LLM to be used downstream.
- Working with langchain agents and tools.
- Using the OpenAI Moderation API to filter explicit content.

For this project, we are using two small samples: `financial_headlines.txt` and `reddit_comments.txt`. These 5-6 line samples are kept short to keep evaluation easy, but keep in mind that this same code and prompt engineering techniques can scale to datasets of much larger size.

### Before you begin

You'll need a developer account with OpenAI.

See getting-started.ipynb for steps on how to create an API key and store it in Workspace. In particular, you'll need to follow the instructions in the "Getting started with OpenAI" and "Setting up Workspace Integrations" sections.

## Task 0: Setup

We need to install a few packages, one of which being the `langchain` package. This is currently being developed quickly, sometimes with breaking changes, so we fix the version.

`langchain` depends on a recent version of `typing_extensions`, so we need to update that package, again fixing the version.

### Instructions

Run the following code to install `openai`, `langchain`, `typing_extensions` and `pandas`.

In [1]:
# Install openai.
!pip install openai

In [2]:
# Install langchain.
!pip install langchain

In [3]:
# Install typing-extensions.
!pip install typing-extensions==4.8.0

In [4]:
# Install pandas.
!pip install pandas

For this project, we need first need to load the openai and os packages to set the API key from the environment variables you just created.

### Instructions

- Import the `os` package.
- Import the `openai` package.
- Set `openai.api_key` to the `OPENAI_API_KEY` environment variable.

In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [7]:
# Import the os package.
import os

# Import the openai package.
import openai

# Set openai.api_key to the OPENAI_API_KEY environment variable.
openai.api_key = os.environ["OPENAI_API_KEY"]

For the `langchain` package, let's start by importing it's `OpenAI` and `ChatOpenAI` class, which are used to interact with completion models and chat completion models respectively.

Completion models, such as the GPT-1, GPT-2, GPT-3 and GPT-3.5, work as an advanced autocomplete model. Given a certain snippet of text as input, they will complete the text until a certain point. This could be either an end-of-sequence token (a natural way of stopping), the model reaching its maximum token limit for outputs and so on.

Chat completion models, such as GPT-3.5-Turbo (the ChatGPT model) and GPT-4, are designed for conversational use. These models are typically more fine-tuned for conversations, keep a prompt/conversation history and allow access to a system message, which we can use as a meta prompt to define a role, a tone of voice, a scope, etc.

Completion models and chat completion models tend to work with different classes and functions in the SDK. For that reason, we will start by importing both classes.

### Instructions

- Import `OpenAI` from `langchain.llms`.
- Import `ChatOpenAI` from `langchain.chat_models`.

<details>
<summary>Code hints</summary>
<p>
    
Remember the syntax for Python imports: `from ... import ...`

</p>
</details>

In [8]:
# Import OpenAI.
from langchain.llms import OpenAI

# Import ChatOpenAI.
from langchain.chat_models import ChatOpenAI

## Task 1: Import the Financial News Headlines Data

A small sample of financial headlines is stored in `financial_headlines.txt`.

Our first step is to read in the text file and store the headlines in a Python list.

### Instructions

Import the text file to a Python list.

- Open `financial_headlines.txt` for reading.
- Read in the lines using the `.readlines()` method. Assign to `headlines`.
- Print the sample headlines.

<details>
<summary>Code hints</summary>
<p>

- A good way of opening (and automatically closing) a file is using: `with open(file_name, "r") as file:`.
- We can then use the `.readlines()` method on the `file` variable.
    
</p>
</details>

In [9]:
# Open the text file and read its lines.
with open('financial_headlines.txt', 'r') as data:
    headlines = data.readlines()

# Print all headlines.
headlines

["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .\n",
 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .\n',
 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .\n',
 'Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .\n',
 "Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .\n",
 'Finnish Outokumpu Technology has been awarded several new grinding technology contracts .']

The headlines seem to a bit of whitespace preceding the punctuation, but this does not influence the performance of our large language model.
You can also see that every headline ends with a new line (`\n`).

We can quickly strip the `\n` from the end of each headline, as this might improve visibility later down the line, when printing these headlines in a dataframe. 

### Instructions

Strip the `\n` character from the end of every news headline.

- Loop through `headlines` and use the `.strip()` method to remove the `\n` character from each line.
- Print the result.

<details>
<summary>Code hints</summary>
<p>

To quickly reassign the adjusted elements of our list, we can make use of Python list comprehensions.
    
For example: `new_list = [f(x) for x in list]`

</p>
</details>

In [10]:
# Strip the new line character from all headlines.
headlines = [line.strip("\n") for line in headlines]

# Print all headlines.
headlines

["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .",
 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .',
 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .',
 'Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .',
 "Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .",
 'Finnish Outokumpu Technology has been awarded several new grinding technology contracts .']

## Task 2: Setting up Prompt Templates

During this code-along we are using the OpenAI API to programmatically make requests to a GPT-model. This allows us to automate calls to the model, as would be the case when implementing generative AI functionalities in an application or data transformation process.

In general, when developing an application, we want our code to be modular, scalable and reusable. How do we this with LLM prompts?

This is where Prompt Templates come into play! It allows for dynamic prompts, with built-in verification tools on whether all inputs are given (this will ease the load on testing). They can easily be saved, versioned and integrated into the code base of an application.

We will set up Prompt Templates (from the `langchain` package) to automatically determine financial sentiment from the headlines and extract relevant company names. 

### Usage of Prompt Templates

A prompt template can have dynamic input, which can be added using `{ }`.

Example: `"Can you give me some suggestions for my trip to {city}?"`

We can then format the prompt template by filling in the `city` variable.

Certain prompts that are often reused programmatically in application processes might be very lengthy and can be carefully designed to meet a specific need. For example, if we want the output of a sentiment analysis by the GPT-model to be limited to either positive, negative or neutral (without anything else in the answer), we need to explicitly tell the model within our prompt. In order to not accidentally forget this in any of the future prompts, it is best practice to design and save a prompt template.

### Types of Prompt Templates

Prompt Templates in Langchain come in two formats:
- PromptTemplate: this is used for completion models.
- ChatPromptTemplate: this is used for chat completion models. On top of the normal input prompt, these can hold a system message (meta prompt) and a conversation history.

Let's start by creating a PromptTemplate and ChatPromptTemplate.

### Instructions

Create a Prompt Template to analyze financial sentiment.
- import `PromptTemplate` from `langchain.prompts`.
- Create a `PromptTemplate` object by using its `.from_template()` method. Assign to `prompt_template`.
- For the template argument, use:

```
"Analyze the following financial headline for sentiment: {headline}"
```

- Format the prompt using its `.format()` method. Let's use our first headline as input. Assign to `formatted_prompt`.
- Print the formatted prompt.

<details>
<summary>Code hints</summary>
<p>

`prompt_template.format()` will have one argument called `headline` (as defined in our `PromptTemplate`), where we pass along the first element of our `headlines` list.

</p>
</details>

In [11]:
# Import the PromptTemplate class.
from langchain.prompts import PromptTemplate

# Create a dynamic template to analyze a single headline.
prompt_template = PromptTemplate.from_template(
    template="Analyze the following financial headline for sentiment: {headline}",
)

# Format the prompt template on the first headline of the dataset.
formatted_prompt = prompt_template.format(headline=headlines[0])

# Print the formatted template.
formatted_prompt

"Analyze the following financial headline for sentiment: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 ."

Now let's set up a `ChatPromptTemplate`, which are compatible with conversational models like GPT-4 and GPT-3.5-Turbo. When using the ChatPromptTemplate, we have the ability to assign a system message, so let's make use of this.

In terms of prompt engineering, what we write in the system can heavily influence the quality of the output. Some things we can do using the system message is:
- Define a role: *"You are a X", "Your role is to do X", ...*
- Define a tone of voice: *"Respond in a formal manner", "Use customer-oriented language", ...*
- Define restrictions on output format: *"The format of the output is X", "The output is strictly limited to X, Y, Z", ...*
- Define a scope: *"Only answer questions on topic X", "If the user questions is not about X, answer with Y", ...*

You will notice some of these tricks applied to the following system message.

### Instructions

- Import `ChatPromptTemplate` from `langchain.prompts`.
- Define a system message as follows and assign to `system_message`.

```
"""You are performing sentiment analysis on news headlines regarding financial analysis. 
This sentiment is to be used to advice financial analysts. 
The format of the output has to be consistent. 
The output is strictly limited to any of the following options: [positive, negative, neutral]."""
```

- Instantiate a new `ChatPromptTemplate` using its `.from_messages()` method. Assign to `chat_template`.
    - This method will take a list of tuples as input. We need two tuples, one for the system message and one for the human message. To distinguish the two, the first element of the tuple is either `"system"` or `"human"`.
    - The second element of the tuple is the actual message, as string. For the system message, you can use the `system_message`variable. For the human message, we can reuse the same message as before (including the input variable `{headlines}`).
    
- Format the template using its `.format_messages()` method. Let's use our first headline again. Assign to `formatted_chat_template`.
- Print the formatted template.

<details>
<summary>Code hints</summary>
<p>

The input for `ChatPromptTemplate.from_messages()` follows this structure:
`[("system", system_message), ("human", input_prompt)]`

</p>
</details>

In [12]:
# Import the ChatPromptTemplate class.
from langchain.prompts import ChatPromptTemplate

# Define the system message.
system_message = """You are performing sentiment analysis on news headlines regarding financial analysis. 
    This sentiment is to be used to advice financial analysts. 
    The format of the output has to be consistent. 
    The output is strictly limited to any of the following options: [positive, negative, neutral]."""

# Initialize a new ChatPromptTemplate with a system message and human message.
chat_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "Analyze the following financial headline for sentiment: {headline}"),
])

# Format the ChatPromptTemplate.
formatted_chat_template = chat_template.format_messages(
    headline=headlines[0]
)

# Print the formatted template.
formatted_chat_template

[SystemMessage(content='You are performing sentiment analysis on news headlines regarding financial analysis. \n    This sentiment is to be used to advice financial analysts. \n    The format of the output has to be consistent. \n    The output is strictly limited to any of the following options: [positive, negative, neutral].'),
 HumanMessage(content="Analyze the following financial headline for sentiment: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .")]

## Task 3: Setting up LLM Chains

We will briefly cover the concept of chains in langchain. LLM Chains are an easy way to combine a model with a prompt template. These chains can be created for both *completion models* and *chat completion models*.

LLM Chains can be used to "chain" prompt flows, by using the output of a previous chain as input for the next.

Let's set up a chain for a completion model first, using the templates that we've just built.

**Instructions**

Create an LLM Chain for a completion model.
- Import `LLMChain` from `langchain.chains`.
- Instantiate a new `LLMChain`. Assign to `completion_chain`. We need to pass along two parameters for its init function:
    - `llm`: Here we can easily pass along a completion model by creating one using `OpenAI()`.
    - `prompt`: Here can we use the `prompt_template` we have created before.
- Run the chain using its `.run()` method.
    - Our prompt template has an input variable called `headline`. This becomes an input parameter for the `.run()` method. Let's pass along the first headline of our dataset here.

In [13]:
# # Import the LLMChain class.
# from langchain.chains import LLMChain

# # Create the LLMChain by combining a completion model and a prompt.
# completion_chain = LLMChain(llm=OpenAI(), prompt=prompt_template)

# # Run the LLMChain.
# completion_chain.run(headline=headlines[0])

In [14]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Import the LLMChain class.
from langchain.chains import LLMChain
from langchain_openai import OpenAI

# Create the LLMChain by combining a completion model and a prompt.
completion_chain = LLMChain(llm=OpenAI(), prompt=prompt_template)

# Run the LLMChain.
completion_chain.run(headline=headlines[0])

/workspaces/datacamp/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nPositive'

Now let's do the same, using a chat completion model.

### Instructions

- Instantiate a new `LLMChain`. Assign to `chat_chain`. We need to pass along two parameters for its init function:
    - `llm`: Here we can easily pass along a completion model by creating one using `ChatOpenAI()`.
    - `prompt`: Here can we use the `chat_template` we have created before.
- Run the chain using the `.run()` method of the chain.

<details>
<summary>Code hints</summary>
<p>

`chat_chain.run()` has two input variables, since `chat_template` has two input variables. Make sure to pass along a value for both `headline` and `system_message`.

</p>
</details>

In [16]:
# Create the LLMChain by combining a chat completion model and a prompt.
chat_chain = LLMChain(llm=ChatOpenAI(), prompt=chat_template)

# Run the LLMChain.
chat_chain.run(headline=headlines[0], system_message=system_message)

/workspaces/datacamp/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'Positive'

## Task 4: Extracting Company Names with the Output Parser

Output parsing is a very useful feature in Langchain when integrating LLM outputs into your application. The output parser can automatically transform the output of the GPT-model to numerous data types, such as lists, datetimes, JSONs and so on.

In this example, we will ask the GPT-model to extract the company name from every headline and instantly assign them to a Python list.

As we want to combine sentiment with the company name later, we will limit the output to one name per headline.

In order to format the output as a Python list, we can make use of the `CommaSeparatedListOutputParser` class in Langchain.

### Instructions

Create an output parser and a formatted prompt template to extract company names from multiple headlines.
- Import `CommaSeparatedListOutputParser` from `langchain.output_parsers`.
- Instantiate a new `CommaSeparatedListOutputParser` and assign to `output_parser`.
- To retrieve the parsing instructions from the output parser, we can use its `.get_format_instructions()` method. Assign this to `format_instructions`.
- Let's instantiate a new `PromptTemplate`. This time we won't use its `.from_template()` method. When calling `PromptTemplate()` with the output parser, we need to pass three arguments:
    - `template`: here we can use the following string; 
```
"List all the company names from the following headlines, limited to one name per headline: {headlines}.\n{format_instructions}"
```

- `input_variables`: This is a list of strings containing the input variables that are required. In our case, this is only `"headlines"`.
- `partial_variables`: Here we pass along a dictionary with the key being `"format_instructions"` and the value being the `format_instructions` variable we created earlier.
- Format the prompt template using the entire `headlines` list.

<details>
<summary>Code hints</summary>
<p>

We can create a new prompt template using `PromptTemplate(template= , input_variables= , partial_variables= )`

</p>
</details>

In [17]:
# Import the CommaSeparatedListOutputParser class.
from langchain.output_parsers import CommaSeparatedListOutputParser

# Instantiate the output parser.
output_parser = CommaSeparatedListOutputParser()

# Get the format instructions from the output parser.
format_instructions = output_parser.get_format_instructions()

# Instantiate a new prompt template with the format instructions.
company_name_template = PromptTemplate(
    template="List all the company names from the following headlines, limited to one name per headline: {headlines}.\n{format_instructions}",
    input_variables=["headlines"],
    partial_variables={"format_instructions": format_instructions}
)

# Format the prompt using all headlines.
formatted_company_name_template = company_name_template.format(headlines=headlines)

Now that we have a template with format instructions ready, let's send it to a GPT-model and look at the output. We want to run these kinds of tasks with the temperature parameter of the large language model set to zero, as this maximizes precision. 

We tend to distinguish tasks that either require precision or creativity. When we are looking for correctness in the answer (e.g. when generating code) we aim for high precision (by lowering temperature) whereas when generating ideas or content, we might prefer more creativity (by increasing temperature). A simplified explanation of the *temperature* of a large language model is its randomness. When temperature is set to 0, we will get the exact same output, given the same inputs.

### Instructions

Create a new Langchain model, send over the template and inspect the parsed output.
- Instantiate a new `OpenAI()` model. Set the temperature to 0. Assign to `model`.
- Run `model()` on the formatted template. Assign to `_output`. The underscore preceding our variable name indicates that this is just a temporary variable, that will likely be overwritten many times.
- Use the `.parse()` method of the output parser on the output of the model. Assign to `company_names`.
- Print the data type of `company_names`.
- Print the company names.

<details>
<summary>Code hints</summary>
<p>

- The temperature of the model can be set to 0 by using `OpenAI(temperature= )`.
- We can get the data type of a variable by using `type(variable)`.

</p>
</details>

In [18]:
# Instantiate a Langchain OpenAI Model object.
model = OpenAI(temperature=0)

# Run the model on the input.
_output = model(formatted_company_name_template)

# Parse the output.
company_names = output_parser.parse(_output)

# Print the data type the parsed output.
print(f"Data type: {type(company_names)}\n")

# Print the output.
print(company_names)

/workspaces/datacamp/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Data type: <class 'list'>

['Aktia Group', 'Vaisala Oyj', 'Orion', 'Tiimari', 'Metso Paper', 'Outokumpu Technology']


## Task 5: Working with Agents and Tools

Leveraging the agents and tools in LangChain is where the framework's value really starts to shine! But before we dive deeper into this concept, we need to understand MRKL prompts.

### What are MRKL Prompts?

MRKL stands for Modular Reasoning, Knowledge and Language prompts. It is a system composed of a set of modules, often accompanied by an agent that decides how to route prompts to the appropriate module (or tools).

These kinds of prompts follow a specific format, which we can force the GPT-model to adhere to by using the system message. It will loop through this format (steps 1-5 below) using recursive requests to the GPT-model until we get our final answer. A commonly used format is the following:
1. Question: the user question (in the first iteration) or follow-up question composed by the GPT-models (in later iterations)
2. Thought: think about what to do as a next step
3. Action: pick a tool from the list of tool names we have provided
4. Action Input: the input for the chosen tool
5. Observation: the output of the tool

### What are Tools and Agents?

We can access (external) tools using the output of the GPT-model. Large language models output only text. In order to call a function (to access a tool) based on the text output of a large language model, we can use agents. They can parse the text output, pick the correct tool and define its input.

The langchain framework has a wide variety of built-in tools, along with the ability to define additional custom tools. A very common use case for tools is accessing document stores or vector databases to ingest information from our own documents. This will be explored more in-depth in future modules.

For now, to have a gentle introduction to tools, we have decided on one that does not require external set up (no API token that needs to be created or external database that needs to be set up). 

In this example, we will make use of the `PythonREPLTool`. This allows the GPT-model to run the Python code that it generates, and can be useful for carrying out an abundance of tasks.

Note: as we introduce recursive prompts using agents, it is best practice to always define a maximum number of output tokens. This ensures our costs will not skyrocket if a prompt loop takes too long.

### Instructions

Before we continue with our financial analysis, let's create a quick example of how code can be ran using a Python agent. In this case, we will ask it to make a calculation (something that most large language models are not trained to do out-of-the-box).
- Import the `create_python_agent` function from `langchain.agents.agent_toolkits`.
- Import the `PythonREPLTool` class from `langchain.tools.python.tool`.
- Create a Python agent by calling the `create_python_agent()` function. Assign to `agent_executor`. This function takes three arguments:
    - `llm`: here we can create a new `OpenAI()` model. Let's set the `temperature` to 0 and `max_tokens` to 1000.
    - `tool`: here we instantiate a new `PythonREPLTool()`.
    - `verbose`: set this to True so that can we see the prompt loop.
- Run the agent using its `.run()` method. As an example, you can ask it: `"What is the square root of 250? Round the answer down to 4 decimals."`

In [19]:
# # Import the necessary classes from langchain.
# from langchain.agents.agent_toolkits import create_python_agent
# from langchain.tools.python.tool import PythonREPLTool

# # Instantiate a Python agent, with the PythonREPLTool.
# agent_executor = create_python_agent(
#     llm=OpenAI(temperature=0, max_tokens=1000),
#     tool=PythonREPLTool(),
#     verbose=True
# )

# # Ask the agent for the solution of a mathematical equation.
# agent_executor.run("What is the square root of 250? Round the answer down to 4 decimals.")

In [20]:
pip install langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [21]:
# Import the necessary classes from langchain.
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

# Instantiate a Python agent, with the PythonREPLTool.
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

# Ask the agent for the solution of a mathematical equation.
agent_executor.run("What is the square root of 250? Round the answer down to 4 decimals.")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I can use the math module to calculate the square root.
Action: Python_REPL
Action Input: import math
Observation: 
Thought: Now that the math module is imported, I can use the sqrt function.
Action: Python_REPL
Action Input: math.sqrt(250)
Observation: 
Thought: The result is a float, so I can use the round function to round it down to 4 decimals.
Action: Python_REPL
Action Input: round(math.sqrt(250), 4)
Observation: 
Thought: I now know the final answer.
Final Answer: 15.8114

> Finished chain.


'15.8114'

Investigate the output above. As we haven't assigned any other tool, the choice of tools for the model was quite limited. Hence under Action, it should list the `Python_REPL` tool.
The Action Input will show the actual code that was generated by the GPT-model and executed by the Agent.

Now let's try to use the same agent to help us in our financial news analysis.

We want to structure our prompt in a clear way, explaining a step by step process. For example:
- First, *Analyze the sentiment...*
- Second, *Load this data into a pandas dataframe.*
- Third, *Save this dataframe to a CSV under the name financial_analysis.csv*
- ...

Lastly, we pass along the headlines (input data) itself.

### Instructions

Ask the agent to extract the company name and sentiment from the headlines and save its output in a `.csv` file called `financial_analysis.csv`.
- Run the agent on the following prompt:
    
    ``` 
    f"""For every of the following headlines, extract the company name and whether the financial sentiment is positive, neutral or negative. 
    Load this data into a pandas dataframe. 
    The dataframe will have three columns: the name of the company, whether the financial sentiment is positive or negative and the headline itself. 
    The dataframe can then be saved in the current working directory under the name financial_analysis.csv.
    If a csv file already exists with the same name, it should be overwritten.

    The headlines are the following:
    {headlines}"""
    ```

In [22]:
# Run the agent
agent_executor.run(f"""For every of the following headlines, extract the company name and whether the financial sentiment is   positive, neutral or negative. 
   Load this data into a pandas dataframe. 
   The dataframe will have three columns: the name of the company, whether the financial sentiment is positive or negative and the headline itself. 
   The dataframe can then be saved in the current working directory under the name financial_analysis.csv.
   If a csv file already exists with the same name, it should be overwritten.

   The headlines are the following:
   {headlines}
   """)



> Entering new AgentExecutor chain...
 I need to import the necessary libraries and create a dataframe to store the extracted data.
Action: Python_REPL
Action Input: import pandas as pd
Observation: 
Thought: I need to create a list of the headlines.
Action: Python_REPL
Action Input: headlines = ["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .", 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .', 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .', 'Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .', "Finnish Metso Paper has bee

'The dataframe containing the extracted data has been saved as financial_analysis.csv in the current working directory.'

Observe the output above. Do you see anything that could be improved? We will come back to this later in this notebook.

For now, let's quickly load our `.csv` file in a dataframe to analyze.

### Instructions

Load the data in a dataframe for evaluation.
- Import `pandas` under its usual alias: `pd`.
- Load the `financial_analysis.csv` file into a dataframe. Assign to `df`.
- Print the dataframe. As our dataframe only contains six rows, we can just print the entire dataframe.

<details>
<summary>Code hints</summary>
<p>

Use the `pd.read_csv(filename)` function to load the `.csv` file to dataframe.

</p>
</details>

In [23]:
# Make the necessary import.
import pandas as pd

# Load the CSV file into a dataframe.
df = pd.read_csv("financial_analysis.csv")

# Print the dataframe.
df.head()

,Company Name,Financial Sentiment,Headline
0,Aktia Group,positive,Finnish Aktia Group 's operating profit rose t...
1,Vaisala Oyj,negative,Finnish measuring equipment maker Vaisala Oyj ...
2,Orion,positive,Finnish pharmaceuticals company Orion reports ...
3,Tiimari,positive,"Tiimari , the Finnish retailer , reported to h..."
4,Metso Paper,positive,Finnish Metso Paper has been awarded a contrac...


When analyzing the output above (looking at the company names and sentiment), you will probably notice some room for improvement. 

Company names and sentiment may not be extracted in a very powerful way. The reason for this is that without further instructions, the GPT-model will use the PythonREPLTool (Python code) to complete its task. Looking back at the output from our last call to the Python agent, we may find that it created rule sets on how to extract the company name or determine the sentiment. These hard-coded rules negate the power of large language models! We will improve on this in Task 7.

Another problem that might arise is that the *sentiment* of a sentence can differ from *financial sentiment*. For example, an aggressive headline complaining about a large corporation making too much profit might result in negative sentiment, while from a financial analysis point of view the sentiment is positive. To steer the GPT-model to our desired outcome, we will now introduce few shot learning.

For example, *Company X was awarded a new contract* might be categorized as a neutral sentence. The sentence itself is simply an objective statement or observation. Nothing is mentioned about whether we like or dislike that particular company because of this. From a financial perspective however, this is considered as something positive. To steer the GPT-model to our desired outcome, we will now introduce few shot learning.

## Task 6: Adding Few Shot Learning

Few shot learning basically comes down to adding some examples into our prompt, in this case, what we consider to be positive or negative headlines. A shot refers to an example given to the model in the input prompt (or sometimes the system message).

We distinguish three categories of contextual learning:
- Few shot leaning (multiple examples)
- Single shot learning (one example)
- Zero shot leaning (no examples)

Few shot learning might take more effort in terms of prompt building, but it will generally yield better results, as the model has a better understanding of our desired outcome.

Let's look at an example of financial sentiment analysis without few shot learning first.

### Instructions

Create a prompt template with output parsing to determine the financial sentiment of all headlines.
- Create a new `PromptTemplate` called `sentiment_template`. Remember the three arguments `template`, `input_variables` and `partial_variables`. Assign to `sentiment_template`.
    - We can reuse the `format_instructions` variable that we have loaded into memory before.
    - As a template, use: 
```
"Get the financial sentiment of each of the following headlines. The output is strictly limited to any of the following options: ['Positive', 'Negative', 'Neutral']: {headlines}.\n{format_instructions}"
```


- Format the template on all headlines. Assign to `formatted_sentiment_template`.
- Run the formatted template through our `model` and assign the result to our temporary variable `_output`.
- Parse the output using the output parser. Assign the result to `sentiments`.
- Print the sentiments.

In [24]:
# Create a new prompt template with output parsing.
sentiment_template = PromptTemplate(
    template="Get the financial sentiment of each of the following headlines. The output is strictly limited to any of the following options: ['Positive', 'Negative', 'Neutral']: {headlines}.\n{format_instructions}",
    input_variables=["headlines"],
    partial_variables={"format_instructions": format_instructions}
)

# Format the prompt template.
formatted_sentiment_template = sentiment_template.format(headlines=headlines)

# Run the model on the formatted prompt template.
_output = model(formatted_sentiment_template)

# Parse the output.
sentiments = output_parser.parse(_output)

# Print the list of sentiments.
sentiments

['Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive']

It is hard to evaluate the sentiments without seeing the associated headline. To make our lives easier, let's write a quick function to easily visualize and interpret the result.

### Instructions

Visualize and interpret the results of the sentiment analysis.
- Write a function called `visualize_sentiments` to visualize both the sentiment and associated headline, for all headlines. 
    - The input for this function should be two lists: one containing all headlines and one containing all sentiments.
    - As a best practice, start with using an `assert` that ensures that both lists are of equal length.
    - There are many ways to create this: simply printing with f-strings, making a dictionary or Dataframe, get creative!
- Call the `visualize_sentiments` function using `headlines` and `sentiments` as input.

<details>
<summary>Code hints</summary>
<p>

- We can assert that both input lists are of equal length by using `assert len(list1) == len(list2)`.
- A very simplistic way of visualizing the sentiments per headline is using f-strings, such as `f"{sentiments[i]}: {headlines[i]}"` in a loop. 

</p>
</details>

In [25]:
# Define a new function with two inputs,
def visualize_sentiments(headlines, sentiments):
    # Assert that both inputs are of equal length
    assert len(headlines) == len(sentiments)

    # Visualize the sentiments and their respective headlines
    for i, _ in enumerate(headlines):
        print(f"{sentiments[i].upper()}: {headlines[i]}")

# Call the function
visualize_sentiments(headlines, sentiments)

POSITIVE: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .
NEGATIVE: Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .
POSITIVE: Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .
POSITIVE: Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .
POSITIVE: Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .
POSITIVE: Finnish Outokumpu Technology has been awarded several new grinding technology contracts .


Now we might see that the financial sentiment is not always correctly assigned, such as a contract being awarded not being recognized as a financially positive headline.
To improve the performance, we will add some examples. Few shot learning can be done by either giving some observations (headlines in this case) accompanied by their ground truth (label) *or* by giving an abstract description of what is seen as positive, negative or neutral.

In this case, we will opt for the later. Here is a prompt you can use for few shot learning:

```
"""
If a company is doing financially better than before, the sentiment is positive. For example, when profits or revenue have increased since the last quarter or year, exceeding expectations, a contract is awarded or an acquisition is announced.
If the company's profits are decreasing, losses are mounting up or overall performance is not meeting expectations, the sentiment is negative.
If nothing positive or negative is mentioned from a financial perspective, the sentiment is neutral.
"""
```

### Instructions

Create and run a prompt template using few shot learning.
- Store the prompt above in a variable called `sentiment_examples`.
- Create a `PromptTemplate` called `sentiment_template` like we did two cells above.
    - In our template, we will add a new input variable called `few_shot_examples`. This can be placed in between the two sentences.
    - Don't forget to add our new input variables to the list of `input_variables`.
    - Reuse the same `format_instructions` as before.
- Format the `sentiment_template`. Remember that you will need to pass both `headlines` and `sentiment_examples`.
- Run the formatted template through our `model` and assign the result to our temporary variable `_output`.
- Parse the output using the output parser. Assign the result to `sentiments`.
- Visualize and interpret the results using your newly created `visualize_sentiments` function.

<details>
<summary>Code hints</summary>
<p>

The `template` we should use could look like this: 

```
"Get the financial sentiment of each of the following headlines. {few_shot_examples} The output is strictly limited to [`Positive`, `Negative`, `Neutral`]: {headlines}.\n{format_instructions}"
```

When formatting the template, we can pass along `sentiment_examples` to the `few_shot_examples` input variable.
    
</p>
</details>

In [26]:
# Store the few shot examples in a variable.
sentiment_examples = """
    If a company is doing financially better than before, the sentiment is positive. For example, when profits or revenue have increased since the last quarter or year, exceeding expectations, a contract is awarded or an acquisition is announced.
    If the company's profits are decreasing, losses are mounting up or overall performance is not meeting expectations, the sentiment is negative.
    If nothing positive or negative is mentioned from a financial perspective, the sentiment is neutral.
"""

# Instantiate a new prompt template with the format instructions.
sentiment_template = PromptTemplate(
    template="Get the financial sentiment of each of the following headlines. {few_shot_examples} The output is strictly limited to any of the following options: ['Positive', 'Negative', 'Neutral']: {headlines}.\n{format_instructions}",
    input_variables=["headlines", "few_shot_examples"],
    partial_variables={"format_instructions": format_instructions}
)

# Format the template.
formatted_sentiment_template = sentiment_template.format(headlines=headlines, few_shot_examples=sentiment_examples)

# Run the model on the formatted template.
_output = model(formatted_sentiment_template)

# Parse the model output.
sentiments = output_parser.parse(_output)

# Visualize the result.
visualize_sentiments(headlines, sentiments)

POSITIVE: Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .
NEGATIVE: Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .
POSITIVE: Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .
POSITIVE: Tiimari , the Finnish retailer , reported to have geenrated quarterly revenues totalling EUR 1.3 mn in the 4th quarter 2009 , up from EUR 0.3 mn loss in 2008 .
POSITIVE: Finnish Metso Paper has been awarded a contract for the rebuild of Sabah Forest Industries ' ( SFI ) pulp mill in Sabah , Malaysia .
POSITIVE: Finnish Outokumpu Technology has been awarded several new grinding technology contracts .


## Task 7: Combining Tools and Output Parsing

As you may have noticed in Task 5, using tools is not a guaranteed success. We can improve the performance by clearly determining which tasks can be completed by the Python tool and which we use the GPT-model itself for.
To maximize the powerful capabilities of the GPT-model, we prefer its use over hard-coded rule sets when it comes to company name extraction or financial sentiment analysis.
However, other (cumbersome) tasks that do not require the ability to handle ambiguity, are often best left to the Python tool.

Let's ask the model to use the existing lists that we got from our templates (`company_names` and `sentiments`), but use the Python tool to neatly place them in a Pandas dataframe and write them locally to a `.csv` file.

Use the following prompt:

```
f"""Create a dataframe with two columns: company_name, sentiment and headline.
                   To fill the dataframe, use the following lists respectively: {str(company_names)}, {str(sentiments)} and {str(headlines)}. 
                   The dataframe can then be saved in the current working directory under the name financial_analysis_with_parsing.csv.
                   If a csv file already exists with the same name, it should be overwritten.
                   """
```

In the prompt above, we pass along lists that were generated by the GPT-model before (when it did not have access to the Python tool). Now we only want to give instructions on tasks that should be carried out using Python code, such as the creation of the dataframe, saving (and overwriting) it, ...

Keep in mind that we can use this same way of working for much more complex tasks, that might encompass extensive coding requirements.

### Instructions

- Run the `agent_executor` on the prompt above.

In [27]:
# Run the agent to create a file with the headlines, company names and sentiments.
agent_executor.run(f"""Create a dataframe with two columns: company_name, sentiment and headline.
To fill the dataframe, use the following lists respectively: {str(company_names)}, {str(sentiments)} and {str(headlines)}. 
The dataframe can then be saved in the current working directory under the name financial_analysis_with_parsing.csv.
If a csv file already exists with the same name, it should be overwritten.
""")



> Entering new AgentExecutor chain...
 I need to import the pandas library to create a dataframe.
Action: Python_REPL
Action Input: import pandas as pd
Observation: 
Thought: I need to create the lists for the columns and data.
Action: Python_REPL
Action Input: company_name = ['Aktia Group', 'Vaisala Oyj', 'Orion', 'Tiimari', 'Metso Paper', 'Outokumpu Technology']
sentiment = ['Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive']
headline = ["Finnish Aktia Group 's operating profit rose to EUR 17.5 mn in the first quarter of 2010 from EUR 8.2 mn in the first quarter of 2009 .", 'Finnish measuring equipment maker Vaisala Oyj HEL : VAIAS said today that its net loss widened to EUR4 .8 m in the first half of 2010 from EUR2 .3 m in the corresponding period a year earlier .', 'Finnish pharmaceuticals company Orion reports profit before taxes of EUR 70.0 mn in the third quarter of 2010 , up from EUR 54.9 mn in the corresponding period in 2009 .', 'Tiimari , the Finnish re

'The dataframe has been successfully created and saved as a csv file in the current working directory under the name financial_analysis_with_parsing.csv.'

If we look at our working directory, we will see a new file pop up, called `financial_analysis_with_parsing.csv`.

Let's analyze it and compare against the output from Task 5.

### Instructions

Load and display the new file.
- Load `financial_analysis_with_parsing.csv` into a dataframe called `df`.
- Print the dataframe.

In [28]:
# Load the CSV file into a dataframe.
df = pd.read_csv("financial_analysis_with_parsing.csv")

# Print the dataframe.
df

,company_name,sentiment,headline
0,Aktia Group,Positive,Finnish Aktia Group 's operating profit rose t...
1,Vaisala Oyj,Negative,Finnish measuring equipment maker Vaisala Oyj ...
2,Orion,Positive,Finnish pharmaceuticals company Orion reports ...
3,Tiimari,Positive,"Tiimari , the Finnish retailer , reported to h..."
4,Metso Paper,Positive,Finnish Metso Paper has been awarded a contrac...
5,Outokumpu Technology,Positive,Finnish Outokumpu Technology has been awarded ...


## Task 8: Using the OpenAI Moderation API

The OpenAI API platform also sports a Moderation API, in addition to their model and embeddings APIs. The Moderation API can check whether the prompt contains explicit content and can flag various categories like hate, violence, sexually explicit content and so on. When we are building an application targeting large user bases, it becomes crucial to leverage the Moderation API and filter our input prompts to avoid the complications associated with unethical LLM usage.

To test the Moderation API, we have a small sample of five comments picked from the `r/WallStreetBets` subreddit, stored in the `reddit_comments.txt` file.

Let's start by reading the text file.

### Content warning

In order to trigger the moderation API, the comments were specifically chosen to be offensive. If you are sensitive to awful content, you may wish to avoid printing and reading the text.

Naturally, neither the project instructor nor DataCamp agrees with the ideas expressed within this text file.

### Instructions

Read the text file and store its lines in a variable called `comments`.
- Open `reddit_comments.txt` as read.
- Use the `.readlines()` method to store its contents in a list called `comments`.
- Optionally print the comments.

<details>
<summary>Code hints</summary>
<p>

Here we can use the same `with open(filename, "r") as file:` structure as in Task 1.

</p>
</details>

In [29]:
# Load the lines of the text file.
with open('reddit_comments.txt', 'r') as data:
    comments = data.readlines()

# Optionally print the comments.
comments

["It's the poors fault for thinking they had a chance in a negative sum gambling casino run by people richer than you who hired Asian quants that are smarter than you.\n",
 "Canada is basically a global real estate investment scheme. It's not even a country, it's a showroom.\n",
 'Lol China not going to make a dent in the global scale. Wake me up when America’s housing market is about to implode that’s when I’m pulling out all my investments. Because the world is going to burn.\n',
 'I would normally have the knee-jerk reaction to seethe at this post but I remind myself that if I had a lot of money I would probably be the snobbiest and stingiest rich person ever. I wouldn’t even help anyone even if they begged me to financially free them from their Wendy’s dumpster obligations\n',
 "I know China will be fine because Peter Zeihan keeps saying China is imploding. If you want to know what the US State Department desperately wants you to believe, just keep up to date with whatever Peter Ze

### Instructions

Analyze a comment using the Moderation API.
- Pick a comment from the dataset (using and index between 0 - 4) and store this in a variable called `comment`.
- Use the API by calling `openai.Moderation.create()`. For the `input` argument, we will pass along the `comment`. Assign to `moderation_output`.
- Print the comment and moderation output.

In [30]:
# Import the openai package.
from openai import OpenAI
client = OpenAI()

# Pick a comment.
comment = comments[0]

# Send the comment to the Moderation API.
moderation_output = client.moderations.create(input=comment)

# Optionally print the comment.
print(comment)

# Print the output.
moderation_output


It's the poors fault for thinking they had a chance in a negative sum gambling casino run by people richer than you who hired Asian quants that are smarter than you.



ModerationCreateResponse(id='modr-9IKAwm60U9Eu3sMs7zCT0rrZ3lZuc', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=True, harassment_threatening=False, hate=False, hate_threatening=False, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_scores=CategoryScores(harassment=0.9698249697685242, harassment_threatening=5.6962966482387856e-05, hate=0.2250063419342041, hate_threatening=1.5335608338773454e-07, self_harm=9.122244648551714e-08, self_harm_instructions=1.5766876515499462e-07, self_harm_intent=2.0730141159219784e-08, sexual=8.78242735780077e-06, sexual_minors=2.3513287317200593e-07, violence=0.00021562023903243244, violence_graphic=4.008067833183304e-07, self-harm=9.122244648551714

We can analyze the output above to determine whether the comment has been deemed explicit or not. The `"flagged"` boolean will show us if any (at least one) category has been flagged, and underneath we can see which categories have been flagged.

## Summary

Congratulations on completing this module! You should be able to get started with basic LangChain projects yourself now. 

You've learned:
- Important prompt engineering tricks and optimizations
- Setting up prompt templates
- Using LLMChains
- Using LangChain output parsing to generate Python objects to be used downstream
- Using LangChain Agents and Tools to add additional functionalities to generative AI projects
- Leveraging the Moderation API to act as a filter of user input

We wish you the best of luck in the following modules!